In [150]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

train_dir = '/kaggle/input/herbarium-2022-fgvc9/train_images/'
test_dir = '/kaggle/input/herbarium-2022-fgvc9/test_images'

with open("/kaggle/input/herbarium-2022-fgvc9/train_metadata.json") as json_file:
    train_meta = json.load(json_file)
with open("/kaggle/input/herbarium-2022-fgvc9/test_metadata.json") as json_file:
    test_meta = json.load(json_file)

In [151]:
image_ids = [image["image_id"] for image in train_meta["images"]]
image_dirs = [train_dir + image['file_name'] for image in train_meta["images"]]
category_ids = [annotation['category_id'] for annotation in train_meta['annotations']]
genus_ids = [annotation['genus_id'] for annotation in train_meta['annotations']]

test_ids = [image['image_id'] for image in test_meta]
test_dirs = [test_dir + image['file_name'] for image in test_meta]

#Create the initial training dataframe with the above defined columns
main_df = pd.DataFrame({
    "image_id" : image_ids,
    "image_dir" : image_dirs,
    "category" : category_ids,
    "genus" : genus_ids})

#Create a testing dataframe
test_df = pd.DataFrame({
    "test_id" : test_ids,
    "test_dir" : test_dirs
})

main_df

,image_id,image_dir,category,genus
0,00000__001,/kaggle/input/herbarium-2022-fgvc9/train_image...,0,1
1,00000__002,/kaggle/input/herbarium-2022-fgvc9/train_image...,0,1
2,00000__003,/kaggle/input/herbarium-2022-fgvc9/train_image...,0,1
3,00000__004,/kaggle/input/herbarium-2022-fgvc9/train_image...,0,1
4,00000__005,/kaggle/input/herbarium-2022-fgvc9/train_image...,0,1
...,...,...,...,...
839767,15504__032,/kaggle/input/herbarium-2022-fgvc9/train_image...,15504,2584
839768,15504__033,/kaggle/input/herbarium-2022-fgvc9/train_image...,15504,2584
839769,15504__035,/kaggle/input/herbarium-2022-fgvc9/train_image...,15504,2584
839770,15504__036,/kaggle/input/herbarium-2022-fgvc9/train_image...,15504,2584


In [152]:
#Add a genus column to the dataframe
genus_map = {genus['genus_id'] : genus['genus'] for genus in train_meta['genera']}
main_df['genus'] = main_df['genus'].map(genus_map)

##Create a family column in the dataframe based on the genus names
    # Step 1: Create dictionary of genus -> family mapping
genus_family_map = {}
for category in train_meta["categories"]:
    genus = category['genus']
    family = category['family']
    genus_family_map[genus] = family

    # Step 2: Create new column with default value of None™
main_df['family'] = None

    # Step 3: Update values in new column based on genus -> family mapping
for i, row in main_df.iterrows():
    genus = row['genus']
    if genus in genus_family_map:
        family = genus_family_map[genus]
        main_df.at[i, 'family'] = family

main_df

,image_id,image_dir,category,genus,family
0,00000__001,/kaggle/input/herbarium-2022-fgvc9/train_image...,0,Abies,Pinaceae
1,00000__002,/kaggle/input/herbarium-2022-fgvc9/train_image...,0,Abies,Pinaceae
2,00000__003,/kaggle/input/herbarium-2022-fgvc9/train_image...,0,Abies,Pinaceae
3,00000__004,/kaggle/input/herbarium-2022-fgvc9/train_image...,0,Abies,Pinaceae
4,00000__005,/kaggle/input/herbarium-2022-fgvc9/train_image...,0,Abies,Pinaceae
...,...,...,...,...,...
839767,15504__032,/kaggle/input/herbarium-2022-fgvc9/train_image...,15504,Zygophyllum,Zygophyllaceae
839768,15504__033,/kaggle/input/herbarium-2022-fgvc9/train_image...,15504,Zygophyllum,Zygophyllaceae
839769,15504__035,/kaggle/input/herbarium-2022-fgvc9/train_image...,15504,Zygophyllum,Zygophyllaceae
839770,15504__036,/kaggle/input/herbarium-2022-fgvc9/train_image...,15504,Zygophyllum,Zygophyllaceae


In [153]:
main_df["family"].value_counts()

Asteraceae           110007
Fabaceae              59152
Poaceae               53547
Cyperaceae            45447
Boraginaceae          23724
                      ...  
Alstroemeriaceae         17
Philesiaceae             17
Velloziaceae             16
Anacampserotaceae        13
Cardiopteridaceae         9
Name: family, Length: 272, dtype: int64

In [154]:
#Filter only the images of plants that are in the Poaceae family
main_df = main_df.loc[main_df['family'] == 'Poaceae']
#main_df = main_df[main_df['family'].isin(['Poaceae', 'Fabaceae'])]
#Reset index
main_df = main_df.reset_index(drop=True)

main_df

,image_id,image_dir,category,genus,family
0,00333__001,/kaggle/input/herbarium-2022-fgvc9/train_image...,333,Agrostis,Poaceae
1,00333__002,/kaggle/input/herbarium-2022-fgvc9/train_image...,333,Agrostis,Poaceae
2,00333__003,/kaggle/input/herbarium-2022-fgvc9/train_image...,333,Agrostis,Poaceae
3,00333__004,/kaggle/input/herbarium-2022-fgvc9/train_image...,333,Agrostis,Poaceae
4,00333__005,/kaggle/input/herbarium-2022-fgvc9/train_image...,333,Agrostis,Poaceae
...,...,...,...,...,...
53542,15501__101,/kaggle/input/herbarium-2022-fgvc9/train_image...,15501,Zuloagaea,Poaceae
53543,15501__103,/kaggle/input/herbarium-2022-fgvc9/train_image...,15501,Zuloagaea,Poaceae
53544,15501__105,/kaggle/input/herbarium-2022-fgvc9/train_image...,15501,Zuloagaea,Poaceae
53545,15501__106,/kaggle/input/herbarium-2022-fgvc9/train_image...,15501,Zuloagaea,Poaceae


In [155]:
main_df["genus"].value_counts()

Muhlenbergia       4228
Paspalum           3124
Poa                2608
Dichanthelium      2474
Sporobolus         2304
                   ... 
Ptilagrostiella      14
Rhipidocladum        11
Dupontia             10
Kalinia              10
Barkworthia           8
Name: genus, Length: 158, dtype: int64

# Model creating

In [156]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.models import resnet50, ResNet50_Weights
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from PIL import Image
import timm

!pip install fvcore

In [157]:
main_df['genus'] = pd.factorize(main_df['genus'])[0]
main_df['category'] = pd.factorize(main_df['category'])[0]

main_df

,image_id,image_dir,category,genus,family
0,00333__001,/kaggle/input/herbarium-2022-fgvc9/train_image...,0,0,Poaceae
1,00333__002,/kaggle/input/herbarium-2022-fgvc9/train_image...,0,0,Poaceae
2,00333__003,/kaggle/input/herbarium-2022-fgvc9/train_image...,0,0,Poaceae
3,00333__004,/kaggle/input/herbarium-2022-fgvc9/train_image...,0,0,Poaceae
4,00333__005,/kaggle/input/herbarium-2022-fgvc9/train_image...,0,0,Poaceae
...,...,...,...,...,...
53542,15501__101,/kaggle/input/herbarium-2022-fgvc9/train_image...,963,157,Poaceae
53543,15501__103,/kaggle/input/herbarium-2022-fgvc9/train_image...,963,157,Poaceae
53544,15501__105,/kaggle/input/herbarium-2022-fgvc9/train_image...,963,157,Poaceae
53545,15501__106,/kaggle/input/herbarium-2022-fgvc9/train_image...,963,157,Poaceae


In [158]:
grouped = main_df.groupby('genus')
# Split the dataset for each class separately
train_dfs = []
test_dfs = []

for name, group in grouped:
    # Split the group into train and test sets
    train, test = train_test_split(group, test_size=0.2, random_state=42)
    # Add the train and test dataframes to the respective lists
    train_dfs.append(train)
    test_dfs.append(test)

# Concatenate the training and evaluation sets for all classes into single DataFrames
train_df_initial = pd.concat(train_dfs)
train_df_initial = train_df_initial.reset_index(drop=True)

test_df = pd.concat(test_dfs)
test_df = test_df.reset_index(drop=True)

In [159]:
grouped2 = train_df_initial.groupby('genus')
# Split the dataset for each class separately
train_dfs = []
val_dfs = []

for name, group in grouped2:
    # Split the group into train and test sets
    train, validation = train_test_split(group, test_size=0.2, random_state=42)
    # Add the train and test dataframes to the respective lists
    train_dfs.append(train)
    val_dfs.append(validation)

# Concatenate the training and evaluation sets for all classes into single DataFrames
train_df = pd.concat(train_dfs)
train_df = train_df.reset_index(drop=True)

val_df = pd.concat(val_dfs)
val_df = val_df.reset_index(drop=True)

In [160]:
batch_size = 32
epochs = 4
IM_SIZE = 224
learning_rate = 1e-4

X_train, Y_train = train_df["image_dir"].values, train_df["genus"].values

X_val, Y_val = val_df["image_dir"].values, val_df["genus"].values

X_test, Y_test  = test_df["image_dir"].values, test_df["genus"].values

Transform = transforms.Compose([
     transforms.Resize((256, 256)),
     transforms.CenterCrop(224),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

In [161]:
class GetData(Dataset):
    def __init__(self, FNames, Labels, Transform):
        self.fnames = FNames
        self.transform = Transform
        self.labels = Labels         
        
    def __len__(self):
        return len(self.fnames)

    def __getitem__(self, index):       
        x = Image.open(self.fnames[index])
    
        if "train" in self.fnames[index]:             
            return self.transform(x), self.labels[index]
                
trainset = GetData(X_train, Y_train, Transform)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)

valset = GetData(X_val, Y_val, Transform)
valloader = DataLoader(valset, batch_size=batch_size, shuffle=True)

testset = GetTestData(X_test, Y_test, Transform)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=True)

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest101', pretrained=True)            

Using cache found in /root/.cache/torch/hub/zhanghang1989_ResNeSt_master


In [162]:
num_classes = train_df['genus'].nunique()
print(num_classes)

158


In [163]:
total_layers = len(list(model.parameters()))

for param in model.parameters():
    param.requires_grad = True
    
n_inputs = model.fc.in_features
last_layer = nn.Linear(n_inputs, num_classes)
model.fc = last_layer 
if torch.cuda.is_available():
    model.cuda()
print(model.fc.out_features)    

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)

158


In [164]:
from tqdm import tqdm
from sklearn.metrics import f1_score

def train(trainloader, model, criterion, optimizer, scaler, device=torch.device("cpu")):
    train_acc = 0.0
    train_loss = 0.0
    y_true = []
    y_pred = []
    for images, labels in tqdm(trainloader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast(enabled=True):
            output = model(images)
            loss = criterion(output, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            acc = ((output.argmax(dim=1) == labels).float().mean())
            train_acc += acc
            train_loss += loss
            y_true += labels.cpu().numpy().tolist()
            y_pred += output.argmax(dim=1).cpu().numpy().tolist()
            
    train_f1 = f1_score(y_true, y_pred, average=None)
    train_f1_avg = f1_score(y_true, y_pred, average='macro')
    return train_acc/len(trainloader), train_loss/len(trainloader), train_f1, train_f1_avg

In [165]:
def evaluate(testloader, model, criterion, device=torch.device("cpu")):
    model.eval()
    eval_acc = 0.0
    eval_loss = 0.0
    y_true = []
    y_pred = []
    for images, labels in tqdm(testloader):
        images = images.to(device)
        labels = labels.to(device)
        with torch.no_grad():
            output = model(images)
            loss = criterion(output, labels)
        acc = ((output.argmax(dim=1) == labels).float().mean())
        eval_acc += acc
        eval_loss += loss
        y_true += labels.cpu().numpy().tolist()
        y_pred += output.argmax(dim=1).cpu().numpy().tolist()
  
    eval_f1 = f1_score(y_true, y_pred, average=None)
    eval_f1_avg = f1_score(y_true, y_pred, average='macro')
    return eval_acc/len(testloader), eval_loss/len(testloader), eval_f1, eval_f1_avg

In [166]:
scaler = torch.cuda.amp.GradScaler(enabled=True)

train_f1_scores = []  # Initialize an empty list to store training F1 scores
val_f1_scores = []  # Initialize an empty list to store validation F1 scores

for epoch in range(epochs):
    train_acc, train_loss, train_f1, train_f1_avg = train(trainloader, model, criterion, optimizer, scaler, device=device)
    eval_acc, eval_loss, eval_f1, eval_f1_avg = evaluate(valloader, model, criterion, device=torch.device("cuda"))

    train_f1_scores.append(train_f1_avg)  # Store the training F1 score for this epoch
    val_f1_scores.append(eval_f1_avg)  # Store the validation F1 score for this epoch

    print(f"Epoch {epoch + 1} | Train Acc: {train_acc*100} | Train Loss: {train_loss} | Train F1 (Avg): {train_f1_avg}")
    print(f"\t Val Acc: {eval_acc*100} | Val Loss: {eval_loss} | Val F1 (Avg): {eval_f1_avg}")
    
    print("F1 score per class (Train):")
    for i, f1 in enumerate(train_f1):
        print(f"Class {i}: {f1}")

    print("\nF1 score per class (Validation):") 
    for i, f1 in enumerate(eval_f1):
        print(f"Class {i}: {f1}")

    print("===="*8)

100%|██████████| 270/270 [04:04<00:00,  1.10it/s]


Epoch 1 | Train Acc: 47.018375396728516 | Train Loss: 2.1482784748077393 | Train F1 (Avg): 0.2510938947347015
	 Val Acc: 64.28406524658203 | Val Loss: 1.2975165843963623 | Val F1 (Avg): 0.42174707907203807
F1 score per class (Train):
Class 0: 0.4334677419354839
Class 1: 0.0
Class 2: 0.5141242937853108
Class 3: 0.0
Class 4: 0.4113475177304964
Class 5: 0.5714285714285715
Class 6: 0.4575163398692811
Class 7: 0.0
Class 8: 0.29411764705882354
Class 9: 0.0
Class 10: 0.6396233248822891
Class 11: 0.12765957446808512
Class 12: 0.40875912408759124
Class 13: 0.0425531914893617
Class 14: 0.38866396761133604
Class 15: 0.0
Class 16: 0.0
Class 17: 0.38181818181818183
Class 18: 0.5241379310344828
Class 19: 0.5351972041937094
Class 20: 0.5341614906832298
Class 21: 0.6408143480368396
Class 22: 0.39292730844793716
Class 23: 0.19354838709677416
Class 24: 0.47115384615384615
Class 25: 0.0
Class 26: 0.1224489795918367
Class 27: 0.41702127659574467
Class 28: 0.4912280701754386
Class 29: 0.0
Class 30: 0.26760

100%|██████████| 270/270 [03:51<00:00,  1.17it/s]


Epoch 2 | Train Acc: 63.257843017578125 | Train Loss: 1.3124102354049683 | Train F1 (Avg): 0.46120584978928836
	 Val Acc: 68.69047546386719 | Val Loss: 1.1023204326629639 | Val F1 (Avg): 0.5337375552125571
F1 score per class (Train):
Class 0: 0.5499541704857928
Class 1: 0.0
Class 2: 0.7071240105540897
Class 3: 0.0
Class 4: 0.49142857142857144
Class 5: 0.7056856187290969
Class 6: 0.71875
Class 7: 0.31884057971014496
Class 8: 0.5977011494252874
Class 9: 0.0
Class 10: 0.76409666283084
Class 11: 0.5757575757575758
Class 12: 0.6666666666666667
Class 13: 0.25396825396825395
Class 14: 0.5360824742268041
Class 15: 0.0
Class 16: 0.1702127659574468
Class 17: 0.5846153846153846
Class 18: 0.703875968992248
Class 19: 0.7114454594304137
Class 20: 0.8181818181818182
Class 21: 0.8041343669250646
Class 22: 0.5824493731918997
Class 23: 0.5365853658536585
Class 24: 0.6594594594594594
Class 25: 0.2608695652173913
Class 26: 0.3125
Class 27: 0.6038961038961039
Class 28: 0.68955223880597
Class 29: 0.0
Class 

100%|██████████| 270/270 [03:51<00:00,  1.16it/s]


Epoch 3 | Train Acc: 76.30110931396484 | Train Loss: 0.7990819811820984 | Train F1 (Avg): 0.663642624126958
	 Val Acc: 71.05985260009766 | Val Loss: 1.0225701332092285 | Val F1 (Avg): 0.5907434971066329
F1 score per class (Train):
Class 0: 0.695731153496821
Class 1: 0.36363636363636365
Class 2: 0.8051282051282052
Class 3: 0.0
Class 4: 0.7979274611398963
Class 5: 0.8187919463087249
Class 6: 0.8043478260869565
Class 7: 0.5517241379310345
Class 8: 0.7932960893854748
Class 9: 0.4444444444444444
Class 10: 0.8534278959810874
Class 11: 0.7105263157894737
Class 12: 0.8202247191011236
Class 13: 0.5753424657534245
Class 14: 0.7432024169184291
Class 15: 0.0
Class 16: 0.5084745762711864
Class 17: 0.7737226277372262
Class 18: 0.82642089093702
Class 19: 0.8392759188151399
Class 20: 0.9285714285714285
Class 21: 0.895410885805763
Class 22: 0.7088607594936709
Class 23: 0.6373626373626373
Class 24: 0.8
Class 25: 0.7222222222222222
Class 26: 0.49382716049382713
Class 27: 0.8012618296529969
Class 28: 0.82

  9%|▉         | 95/1068 [01:56<19:50,  1.22s/it]


KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(train_f1_scores, label="Train F1 (Avg)")
plt.plot(val_f1_scores, label="Validation F1 (Avg)")
plt.xlabel("Epoch")
plt.ylabel("F1 Score")
plt.title(f"F1 Scores Over Epochs")
plt.legend()
plt.xticks(range(len(train_f1_scores)), range(1, len(train_f1_scores) + 1))
plt.show()

In [167]:
from sklearn.metrics import classification_report

test_acc = 0.0
y_true = []
y_pred = []

model = model.to(device)

# Set the model to evaluation mode
model.eval()

# Loop through the test set
with torch.no_grad():
    for images, labels in tqdm(testloader, desc="Testing"):
        images = images.to(device)
        labels = labels.to(device)

        # Make predictions
        output = model(images)

        # Calculate accuracy
        acc = (output.argmax(dim=1) == labels).float().mean()
        test_acc += acc

        # Store true labels and predicted labels for other metrics
        y_true += labels.cpu().numpy().tolist()
        y_pred += output.argmax(dim=1).cpu().numpy().tolist()

# Calculate the final test accuracy
test_acc = test_acc / len(testloader)
print(f"Test accuracy: {test_acc:.2%}")

# Calculate the classification report, which includes the F1 score for all classes
report = classification_report(y_true, y_pred, output_dict=True)

# Print the classification report
for class_label, metrics in report.items():
    if class_label.isdigit():
        print(f"Class {class_label}: F1 score = {metrics['f1-score']:.2f}")

avg_f1 = f1_score(y_true, y_pred, average='macro')
print(f"Average F1 score: {avg_f1:.2f}")

Testing: 100%|██████████| 337/337 [05:03<00:00,  1.11it/s]

Test accuracy: 74.00%
Class 0: F1 score = 0.67
Class 1: F1 score = 0.00
Class 2: F1 score = 0.79
Class 3: F1 score = 0.00
Class 4: F1 score = 0.75
Class 5: F1 score = 0.79
Class 6: F1 score = 0.86
Class 7: F1 score = 0.40
Class 8: F1 score = 0.72
Class 9: F1 score = 0.40
Class 10: F1 score = 0.87
Class 11: F1 score = 0.74
Class 12: F1 score = 0.79
Class 13: F1 score = 0.62
Class 14: F1 score = 0.53
Class 15: F1 score = 0.00
Class 16: F1 score = 0.44
Class 17: F1 score = 0.74
Class 18: F1 score = 0.85
Class 19: F1 score = 0.83
Class 20: F1 score = 0.90
Class 21: F1 score = 0.90
Class 22: F1 score = 0.65
Class 23: F1 score = 0.69
Class 24: F1 score = 0.83
Class 25: F1 score = 0.50
Class 26: F1 score = 0.92
Class 27: F1 score = 0.81
Class 28: F1 score = 0.76
Class 29: F1 score = 0.00
Class 30: F1 score = 0.82
Class 31: F1 score = 0.73
Class 32: F1 score = 0.60
Class 33: F1 score = 0.83
Class 34: F1 score = 0.80
Class 35: F1 score = 0.96
Class 36: F1 score = 0.84
Class 37: F1 score = 0.58



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
scaler = torch.cuda.amp.GradScaler(enabled=True)

train_f1_scores = []  # Initialize an empty list to store training F1 scores
val_f1_scores = []  # Initialize an empty list to store validation F1 scores

train_acc, train_loss, train_f1, train_f1_avg = train(trainloader, model, criterion, optimizer, scaler, device=device)
eval_acc, eval_loss, eval_f1, eval_f1_avg = evaluate(valloader, model, criterion, device=torch.device("cuda"))

train_f1_scores.append(train_f1_avg)  # Store the training F1 score for this epoch
val_f1_scores.append(eval_f1_avg)  # Store the validation F1 score for this epoch

print(f"Epoch {epoch + 1} | Train Acc: {train_acc*100} | Train Loss: {train_loss} | Train F1 (Avg): {train_f1_avg}")
print(f"\t Val Acc: {eval_acc*100} | Val Loss: {eval_loss} | Val F1 (Avg): {eval_f1_avg}")

print("F1 score per class (Train):")
for i, f1 in enumerate(train_f1):
    print(f"Class {i}: {f1}")

print("\nF1 score per class (Validation):") 
for i, f1 in enumerate(eval_f1):
    print(f"Class {i}: {f1}")

print("===="*8)

 72%|███████▏  | 194/270 [02:52<01:05,  1.16it/s]

In [1]:
from sklearn.metrics import classification_report

test_acc = 0.0
y_true = []
y_pred = []

model = model.to(device)

# Set the model to evaluation mode
model.eval()

# Loop through the test set
with torch.no_grad():
    for images, labels in tqdm(testloader, desc="Testing"):
        images = images.to(device)
        labels = labels.to(device)

        # Make predictions
        output = model(images)

        # Calculate accuracy
        acc = (output.argmax(dim=1) == labels).float().mean()
        test_acc += acc

        # Store true labels and predicted labels for other metrics
        y_true += labels.cpu().numpy().tolist()
        y_pred += output.argmax(dim=1).cpu().numpy().tolist()

# Calculate the final test accuracy
test_acc = test_acc / len(testloader)
print(f"Test accuracy: {test_acc:.2%}")

# Calculate the classification report, which includes the F1 score for all classes
report = classification_report(y_true, y_pred, output_dict=True)

# Print the classification report
for class_label, metrics in report.items():
    if class_label.isdigit():
        print(f"Class {class_label}: F1 score = {metrics['f1-score']:.2f}")

avg_f1 = f1_score(y_true, y_pred, average='macro')
print(f"Average F1 score: {avg_f1:.2f}")

NameError: name 'model' is not defined

In [ ]:
scaler = torch.cuda.amp.GradScaler(enabled=True)

train_f1_scores = []  # Initialize an empty list to store training F1 scores
val_f1_scores = []  # Initialize an empty list to store validation F1 scores

train_acc, train_loss, train_f1, train_f1_avg = train(trainloader, model, criterion, optimizer, scaler, device=device)
eval_acc, eval_loss, eval_f1, eval_f1_avg = evaluate(valloader, model, criterion, device=torch.device("cuda"))

train_f1_scores.append(train_f1_avg)  # Store the training F1 score for this epoch
val_f1_scores.append(eval_f1_avg)  # Store the validation F1 score for this epoch

print(f"Epoch {epoch + 1} | Train Acc: {train_acc*100} | Train Loss: {train_loss} | Train F1 (Avg): {train_f1_avg}")
print(f"\t Val Acc: {eval_acc*100} | Val Loss: {eval_loss} | Val F1 (Avg): {eval_f1_avg}")

print("F1 score per class (Train):")
for i, f1 in enumerate(train_f1):
    print(f"Class {i}: {f1}")

print("\nF1 score per class (Validation):") 
for i, f1 in enumerate(eval_f1):
    print(f"Class {i}: {f1}")

print("===="*8)

In [ ]:
from sklearn.metrics import classification_report

test_acc = 0.0
y_true = []
y_pred = []

model = model.to(device)

# Set the model to evaluation mode
model.eval()

# Loop through the test set
with torch.no_grad():
    for images, labels in tqdm(testloader, desc="Testing"):
        images = images.to(device)
        labels = labels.to(device)

        # Make predictions
        output = model(images)

        # Calculate accuracy
        acc = (output.argmax(dim=1) == labels).float().mean()
        test_acc += acc

        # Store true labels and predicted labels for other metrics
        y_true += labels.cpu().numpy().tolist()
        y_pred += output.argmax(dim=1).cpu().numpy().tolist()

# Calculate the final test accuracy
test_acc = test_acc / len(testloader)
print(f"Test accuracy: {test_acc:.2%}")

# Calculate the classification report, which includes the F1 score for all classes
report = classification_report(y_true, y_pred, output_dict=True)

# Print the classification report
for class_label, metrics in report.items():
    if class_label.isdigit():
        print(f"Class {class_label}: F1 score = {metrics['f1-score']:.2f}")

avg_f1 = f1_score(y_true, y_pred, average='macro')
print(f"Average F1 score: {avg_f1:.2f}")